#### 한국어 코퍼스 전처리
- 데이터셋: Korpora에서 로딩
- 형태소분석기 설정
- 단어사전 생성

[1] 모듈 로딩 및 데이터 준비<hr>

In [1]:
# 모듈 로딩
from Korpora import Korpora     #한국어 데이터셋
from konlpy.tag import *        # 형태소 분석기
import spacy                    # 형태소 분석기

import pandas as pd
import numpy as np
from torchtext.data.utils import get_tokenizer  #자연어 처리 모듈
from torchtext.data import *
from torchtext.vocab import *                   #단어사전
from torch.utils.data import Dataset, DataLoader

c:\Users\KDP-25\anaconda3\envs\NLP\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\KDP-25\anaconda3\envs\NLP\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\KDP-25\anaconda3\envs\NLP\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHT

In [2]:
nsmc= Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-25\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [3]:
nsmc.train[1], nsmc.test[1]

(LabeledSentence(text='흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', label=1),
 LabeledSentence(text='GDNTOPCLASSINTHECLUB', label=0))

In [4]:
# test 데이터셋을 DataFrame으로 로딩
nsmcDF=pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


[2] 텍스트 데이터 전처리 <hr>
- 토큰화/정제 (불용어, 구두점, 오타 등 처리)
- 단어사전 생성
- 문장 수치화

In [5]:
# 단어 사전을 생성시 활용할 데이터셋

class TextDataset(Dataset):
    def __init__(self, feature, label):
        super().__init__()
        self.feature= feature
        self.label= label
        self.length=feature.shape[0]

    
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.feature[index], self.label[index]

In [6]:
# DS 인스턴스 생성
nsmcDS=TextDataset(nsmcDF['text'], nsmcDF['label'])

In [7]:
for f, l in nsmcDS:
    print(f,l)
    break

굳 ㅋ 1


[2] 텍스트 데이터 전처리 <hr>
- 토큰화/정제 (불용어, 구두점, 오타 등 처리)
- 단어사전 생성
- 문장 수치화

In [8]:
# tokenizer 생성
# () 안에 함수, 스플릿 함수 등 or callable func
okt_tokenizer=get_tokenizer(Okt().morphs)
spacy_tokenizer=get_tokenizer('spacy', language='ko_core_news_lg')

In [34]:
# DS에서 처리한 데이터셋만 로딩ㅇ해서 토큰화 진행하는 반복자 함수
def generateToken(dataset):
    for text, label in dataset:
        tokens= okt_tokenizer(text)   #불용어 처리하려면 여기서도 가능
        yield tokens

In [35]:
# pad, oov 선언
UNK_TOKEN='<UNK>'
PAD_TOKEN='<UNK>'

In [36]:
# 단어 사전 생성
nsmcVocab= build_vocab_from_iterator(iterator=generateToken(nsmcDS), 
                                     specials=['<UNK>', '<PAD>'])
nsmcVocab.set_default_index(nsmcVocab['<UNK>']) #OOV 인덱스 설정

KeyError: 50000